In [1]:
import pandas as pd
import folium
import branca.colormap as cm
import json


def getKarte(geoDaten, kreisliste, colorstep, key_on ):

    #Kreisnummern so verändern, dass sie mit den GoJson Nummern entsprechen
    kreisliste = kreisliste.astype({'kreisnummer' : 'float'})       #get rid of 0 in front
    kreisliste = kreisliste.astype({'kreisnummer' : 'int'})       #get rid of 0 in front
    kreisliste = kreisliste.astype({'kreisnummer' : 'str'})       #only strings can be verglichen werden to show results
    #kreisliste['kreisnummer'] = kreisliste['kreisnummer'] + '000'         #add 000

    #First determine your maximum and minimum values - this becomes the basis for the steps in the map
    vmax = kreisliste[key_on].max()  
    vmin = kreisliste[key_on].min() 
    #Farbverlauf einstellen -> von weiß nach blau zu Lila
    colormap = cm.LinearColormap(colors=['#ffffff', '#61abff', '#b894ff', '#a200ff', '#d000ff'], 
                             vmin=vmin,
                             vmax=vmax)

    #rückgabe des höchsten und mittelwerts in der Tabelle 
    print("max : " +str(vmax))
    print("mean: " +str(kreisliste[key_on].mean()))

    #colormap so einstellen, dass überhang besteht. so werden die Daten von kleineren Landkreisen sichtbar
    colormap = colormap.to_step(len(colorstep),
                data= colorstep,
                method='quantiles',
                round_method='float')

    #karte erstellen - mittelpunkt ist mittelpunkt von deutschland
    map = folium.Map(location = [51.1657,10.4515], zoom_start=6.3)

    #Choropleth karte auf Karte auflegen
    cp = folium.Choropleth(
        geo_data=geoDaten,  #Daten für Landkreisgrenzen
        name="choropleth",  
        data= kreisliste,   #kreisliste als Datensatz pbergeben
        columns=["kreisnummer", key_on],    #zeile kreisnummern zum vergleich mit GeoJson und key_on anzuzeigende Werte
        key_on = 'properties.SN_KRG',       #Ort der Kreisnummern in GeoJson
        line_weight = 0.1,
        fill_opacity= 0.1,
    ).add_to(map)

    #remove one of the two legends
    cp._children
    for key in cp._children:
        if key.startswith('color_map'):
            del(cp._children[key])
        
    #add new legend
    colormap.add_to(map)
    #schreiben der daten in die GEOJSON Datei für tooltips und zum abrufen der Werte für die Colormap
    for s in cp.geojson.data['features']:   #iterieren durch alle json features
        try: 
            row = kreisliste.loc[s['properties']['SN_KRG'] == kreisliste['kreisnummer']] #row index finden wo kreisnummer und SNKRG matcht
            s['properties'][key_on] = str(kreisliste.at[row.index[0], key_on]) #schreiben in die JSON
        except: pass

    #print(cp.geojson.data)
    with open('example.json', 'w') as file:
    # Write the text stored in the variable to the file
        json.dump(cp.geojson.data, file, indent=2)
    #set style 
    highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

    style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.7, 
                            'weight': 0.2,
                            'fillColor': colormap(float(x['properties'][key_on]))       #essentieller part -  ruft colormap mit wert auf
                            }

    #Zusammenfügen aller zuvor festegelgten Personalisierungen 
    NIL = folium.features.GeoJson(
        geoDaten,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(         #tooltops sind zuvor in GeoJson List geschrieben worden
            fields=['KRG', 'SN_KRG',key_on],            #angezeigt werden kreisnamen, Kreisnummern und der Wert von key_on
            aliases=['Kreisname: ', 'Kreisnummer: ', key_on],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
           

    map.add_child(NIL)

    return map

In [3]:
import psycopg2
import geoplotlib
from geoplotlib.utils import BoundingBox, DataAccessObject
from geoplotlib.colors import ColorMap
from pandas import DataFrame
import json

# Connect database with the set credentials in PG
db_connection = psycopg2.connect(
    user="tester",
    password="1234",
    host="localhost",
    port="5432",
    database="ThesisTry1"
)

#updated query
query = """
SELECT 
    we."Landkreis",
    SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 5) AS Kreisnummer,
    SUM(we."Bruttoleistung") AS total_installierte_leistung
FROM 
    wind_extended AS we
GROUP BY 
    we."Landkreis", SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 5);
"""
# Create a cursor and execute the query
cursor = db_connection.cursor()
cursor.execute(query)

# Fetch all the rows from the result
results = cursor.fetchall()

kreisliste = DataFrame(results)
kreisliste.columns = ['landkreis','kreisnummer','InstalledPower']

# Drop rows with missing longitude or latitude values
kreisliste = kreisliste.dropna()


In [4]:
#Kreisnummern so verändern, dass sie mit den GoJson Nummern entsprechen
kreisliste = kreisliste.astype({'kreisnummer' : 'float'})       #get rid of 0 in front
kreisliste = kreisliste.astype({'kreisnummer' : 'int'})       #get rid of 0 in front
kreisliste = kreisliste.astype({'kreisnummer' : 'str'})       #only strings can be verglichen werden to show results
kreisliste['kreisnummer'] = kreisliste['kreisnummer'] + '000'         #add 000
#kreistliste = kreisliste.drop_duplicates(subset="kreisnummer")

display(kreisliste)
print("Minimum Installed Power: ", kreisliste['InstalledPower'].min())
#kreisliste['kreisnummer'].to_csv('dbdata.csv', index=False)
display(kreisliste.sort_values("InstalledPower",ascending=False).head(10))

,landkreis,kreisnummer,InstalledPower
1,Uelzen,3360000,465739.6
2,Verden,3361000,210063.5
3,Schaumburg,3257000,50150.0
4,Cottbus,12052000,64000.0
5,Fürth,9573000,52540.0
...,...,...,...
352,Rotenburg (Wümme),3357000,656006.8
353,Hochtaunuskreis,6434000,33480.0
354,Oder-Spree,12067000,516564.0
355,Rheingau-Taunus-Kreis,6439000,63250.0


Minimum Installed Power:  0.4


,landkreis,kreisnummer,InstalledPower
334,Nordfriesland,1054000,2713746.10
168,Dithmarschen,1051000,2499141.64
166,Uckermark,12073000,1762002.40
280,Emsland,3454000,1491424.20
178,Paderborn,5774000,1487057.40
248,Schleswig-Flensburg,1059000,1387007.40
6,Prignitz,12070000,1345191.00
333,Aurich,3452000,1131170.30
340,Stendal,15090000,1074095.00
59,Höxter,5762000,958697.80


In [ ]:
with open('website/LandkreiseALL.json',encoding='utf8') as handle:
    country_geo = json.loads(handle.read())

#erstellen eines neuen DF für die Landkreisliste 
struktur = {'kreisnummer' : []}
kreislist = DataFrame(struktur)

#beschreiben der kreisliste mit entsprechenden Namen aus der JSON Datei
j = 0
for i in country_geo['features']:
    kreislist.at[j,'kreisnummer'] =  i['properties']['SN_KRG']      #zuweisung vom Kenncode -- dividieren durch 1000 weil Stammdaten aus JSON aus irgendeinem Grund drei Nullen angehängt haben
    j = j+1
#kreislist.to_csv('jsoncompare.csv', index=False)    
kreislist['kreisnummer'].to_csv('geojsonkreisnummer.csv', index=False)

In [ ]:

# Merge based on the "landkreis" column and select the desired columns
merged_df = DataFrame.merge(kreislist, kreisliste, on='kreisnummer', how='outer')
merged_df['InstalledPower'].fillna(0, inplace=True) #fillna function puts o instead of Null to a district. 
merged_df = merged_df.drop_duplicates(subset=["kreisnummer"])
merged_df['InstalledPower'].round(2) # Todo: For some reason it is not working. Get back to it later. 
merged_df['InstalledPower'].to_csv('mergedcsvkreisnummer.csv', index=False)
#print(DataFrame.merge([kreisliste,merged_df]).drop_duplicates(keep=False))
colorStep =  [ 250, 500, 1000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000,100000, 200000, 300000, 400000, 500000, 1000000, 1500000, 2000000, 2500000, 3000000]


map = getKarte(geoDaten = country_geo ,kreisliste = merged_df, colorstep= colorStep, key_on = 'InstalledPower')

map